In [39]:
from bs4 import BeautifulSoup as bs
import lxml

from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
import time
import requests
from selenium.webdriver.common.keys import Keys

path = './chromedriver' # 크롬드라이브 경로 지정  ## 맥과 리눅스는 exe.를 붙이지 않습니다.



In [89]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source
parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_0")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_0")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_0")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_0")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_0")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)

daily_boxoffice_html = driver.page_source
driver.quit()

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_0"})

movie_list = daily_boxoffice.find_all("tr")
#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 


daily_boxoffice1 = pd.DataFrame(daily_movies) 

daily_boxoffice1

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice1['순위 변동'] = daily_boxoffice1['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice1 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,장사리 : 잊혀진 영웅들,0,2019-09-25,"1,730,195,400",33.7%,"201,973","497,845","1,064","4,814"
1,양자물리학,1,2019-09-25,"964,521,750",18.8%,"110,099","276,855",772,"2,728"
2,나쁜 녀석들: 더 무비,-1,2019-09-11,"957,282,740",18.7%,"109,811","4,353,000","1,000","4,024"
3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"467,303,790",9.1%,"52,383","154,415",678,"1,931"
4,애드 아스트라,0,2019-09-19,"176,380,740",3.4%,"19,956","470,595",493,919
5,퍼펙트맨,1,2019-10-02,"147,407,700",2.9%,"17,495","74,204",481,723
6,타짜: 원 아이드 잭,-1,2019-09-11,"107,076,860",2.1%,"11,728","2,202,591",405,907
7,더 룸,-1,2019-09-25,"89,743,700",1.7%,"10,108","36,458",163,496
8,예스터데이,-1,2019-09-18,"72,460,220",1.4%,"8,237","285,543",317,470
9,극장판 헬로카봇 : 달나라를 구해줘!,7,2019-09-04,"51,966,600",1.0%,"6,644","569,399",172,241


In [90]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source
parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_112")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_112")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_112")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_112")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_112")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)

daily_boxoffice_html = driver.page_source
driver.quit()


parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_112"})

movie_list = daily_boxoffice.find_all("tr")



#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
print(daily_movies)

daily_boxoffice2 = pd.DataFrame(daily_movies) 

daily_boxoffice2

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice2['순위 변동'] = daily_boxoffice2['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice2 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

{'영화명': ['장사리 : 잊혀진 영웅들', '나쁜 녀석들: 더 무비', '양자물리학', '원스 어폰 어 타임... 인 할리우드', '애드 아스트라', '타짜: 원 아이드 잭', '더 룸', '예스터데이', '레플리카', '힘을 내요, 미스터 리', '매트릭스', '47미터 2', '메기', '가장 보통의 연애', '벌새', '미드 90', '벨 칸토', '교회오빠', '다운 어 다크 홀', '뱅드림! 필름 라이브', '마왕의 딸 이리샤', '전원사형', '세계는 오늘부터 너의 것', '터미네이터2 3D', '아워 바디', '슈퍼 햄찌', '극장판 헬로카봇 : 달나라를 구해줘!', '100일 동안 100가지로 100퍼센트 행복찾기', '천지창조 / 더 바이블', '레드슈즈', '사이몬과 타카시', '크리스마스 호러 스토리', '리틀 포니 프린세스', '우리집', '뷰티풀 보이', '나, 다니엘 블레이크', '변신', '고뇌와 환희', '그것: 두 번째 이야기', '내 완벽한 남사친의 비밀', '발명왕 볼트', '내 여자의 남자들', '미드소마 감독판', '와인스타인', '엑시트', '알라딘', '비뚤어진 집', '나의 특별한 형제', '애월', '맛있는엄마들'], '순위 변동': ['동일', '동일', '동일', '동일', '동일', '동일', '동일', '1상승', '1하락', '동일', '1상승', '1하락', '동일', '6상승', '6상승', '2하락', '1상승', '6상승', '동일', '4하락', '1상승', '6상승', '5상승', '4하락', '동일', '2하락', '8상승', '3상승', 'New', '8상승', '2상승', '1상승', '2하락', '0상승', '4상승', 'New', '0하락', '2하락', '2상승', '6하락', '2하락', 'New', 'New', '4하락', '1하락', '9상승', '5상승', '7상승', '3하락', 'New'], '개봉일': ['2019-09-25', '2019-09-11', '2019-09-

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,장사리 : 잊혀진 영웅들,0,2019-09-25,"769,141,540",30.3%,"91,325","295,872",979,"4,579"
1,나쁜 녀석들: 더 무비,0,2019-09-11,"505,429,190",19.9%,"59,039","4,243,189",921,"3,806"
2,양자물리학,0,2019-09-25,"475,273,640",18.7%,"54,820","166,756",658,"2,689"
3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"251,645,530",9.9%,"28,391","102,032",668,"1,848"
4,애드 아스트라,0,2019-09-19,"107,790,040",4.2%,"12,135","450,639",489,"1,066"
5,타짜: 원 아이드 잭,0,2019-09-11,"73,243,300",2.9%,"8,274","2,190,863",429,"1,025"
6,더 룸,0,2019-09-25,"56,364,900",2.2%,"6,356","26,350",157,562
7,예스터데이,1,2019-09-18,"47,211,980",1.9%,"5,605","277,306",337,525
8,레플리카,-1,2019-09-25,"43,191,260",1.7%,"5,060","18,625",339,774
9,"힘을 내요, 미스터 리",0,2019-09-11,"23,519,940",0.9%,"3,390","1,158,974",222,319


In [91]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_240")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_240")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_240")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_240")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_240")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
# driver.quit()
daily_boxoffice_html = driver.page_source


parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_240"})

movie_list = daily_boxoffice.find_all("tr")


#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 


daily_boxoffice3 = pd.DataFrame(daily_movies) 

daily_boxoffice3

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice3['순위 변동'] = daily_boxoffice3['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice3 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,장사리 : 잊혀진 영웅들,0,2019-09-25,"521,422,010",27.4%,"66,273","204,547",977,"4,390"
1,나쁜 녀석들: 더 무비,0,2019-09-11,"366,659,350",19.3%,"46,264","4,184,150",891,"3,488"
2,양자물리학,0,2019-09-25,"334,299,980",17.6%,"40,798","111,936",669,"2,665"
3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"181,361,350",9.5%,"22,376","73,641",640,"1,716"
4,애드 아스트라,0,2019-09-19,"82,494,370",4.3%,"9,950","438,504",484,"1,103"
5,타짜: 원 아이드 잭,0,2019-09-11,"62,133,940",3.3%,"7,722","2,182,589",424,"1,018"
6,더 룸,0,2019-09-25,"41,016,200",2.2%,"4,938","19,994",144,572
7,레플리카,1,2019-09-25,"38,023,670",2.0%,"4,831","13,565",344,838
8,예스터데이,-1,2019-09-18,"33,855,500",1.8%,"4,327","271,701",338,541
9,"힘을 내요, 미스터 리",0,2019-09-11,"24,129,260",1.3%,"3,710","1,155,584",215,305


In [92]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_357")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_357")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_357")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_357")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_357")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_357"})

movie_list = daily_boxoffice.find_all("tr")


#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 


daily_boxoffice4 = pd.DataFrame(daily_movies) 

daily_boxoffice4

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice4['순위 변동'] = daily_boxoffice4['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice4 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,장사리 : 잊혀진 영웅들,7,2019-09-25,"770,367,470",31.0%,"129,938","138,274","1,065","4,381"
1,나쁜 녀석들: 더 무비,-1,2019-09-11,"531,802,230",21.4%,"89,092","4,137,886",881,"3,229"
2,양자물리학,4,2019-09-25,"323,242,370",13.0%,"53,451","71,138",731,"2,675"
3,원스 어폰 어 타임... 인 할리우드,7,2019-09-25,"289,132,600",11.6%,"47,987","51,265",670,"1,648"
4,애드 아스트라,-3,2019-09-19,"118,937,110",4.8%,"19,165","428,554",520,"1,159"
5,타짜: 원 아이드 잭,-3,2019-09-11,"88,376,020",3.6%,"14,507","2,174,867",516,"1,083"
6,더 룸,7,2019-09-25,"60,565,900",2.4%,"9,912","15,056",159,587
7,예스터데이,-3,2019-09-18,"55,069,000",2.2%,"8,953","267,374",374,607
8,레플리카,New,2019-09-25,"52,701,990",2.1%,"8,544","8,734",360,871
9,"힘을 내요, 미스터 리",-4,2019-09-11,"34,873,770",1.4%,"5,965","1,151,874",256,359


In [93]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_467")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_467")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_467")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_467")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_467")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
# driver.quit()
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_467"})

movie_list = daily_boxoffice.find_all("tr")


#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 


daily_boxoffice5 = pd.DataFrame(daily_movies) 

daily_boxoffice5

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice5['순위 변동'] = daily_boxoffice5['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice5 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,나쁜 녀석들: 더 무비,0,2019-09-11,"526,512,270",32.3%,"65,944","4,048,794","1,114","5,235"
1,애드 아스트라,0,2019-09-19,"217,903,930",13.4%,"26,323","409,389",787,"3,179"
2,타짜: 원 아이드 잭,0,2019-09-11,"181,626,290",11.1%,"22,523","2,160,360",777,"2,661"
3,퍼펙트맨,2,2019-10-02,"159,523,000",9.8%,"21,932","55,272",147,164
4,예스터데이,-1,2019-09-18,"117,676,790",7.2%,"14,971","258,421",630,"2,161"
5,"힘을 내요, 미스터 리",-1,2019-09-11,"100,037,300",6.1%,"13,612","1,145,909",584,"1,642"
6,양자물리학,2,2019-09-25,"49,990,000",3.1%,"8,570","17,687",64,65
7,장사리 : 잊혀진 영웅들,5,2019-09-25,"33,219,000",2.0%,"3,644","8,336",17,17
8,가장 보통의 연애,New,2019-10-02,"30,123,000",1.8%,"3,347","3,748",15,18
9,그것: 두 번째 이야기,-1,2019-09-04,"28,255,000",1.7%,"3,278","588,985",109,160


In [94]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
# driver.quit()
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_572"})

movie_list = daily_boxoffice.find_all("tr")


#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 


daily_boxoffice6 = pd.DataFrame(daily_movies) 

daily_boxoffice6

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice6['순위 변동'] = daily_boxoffice6['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice6 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,나쁜 녀석들: 더 무비,0,2019-09-11,"609,349,940",41.4%,"75,648","3,982,850","1,084","5,373"
1,애드 아스트라,0,2019-09-19,"250,433,040",17.0%,"30,128","383,066",799,"3,291"
2,타짜: 원 아이드 잭,0,2019-09-11,"210,394,610",14.3%,"26,053","2,137,837",748,"2,713"
3,예스터데이,0,2019-09-18,"131,507,040",8.9%,"16,466","243,450",614,"2,277"
4,"힘을 내요, 미스터 리",0,2019-09-11,"95,046,390",6.5%,"12,712","1,132,297",558,"1,668"
5,퍼펙트맨,2,2019-10-02,"23,418,000",1.6%,"2,640","33,340",13,13
6,더 룸,7,2019-09-25,"18,560,000",1.3%,"2,320","4,848",11,11
7,벌새,-1,2019-08-29,"15,188,600",1.0%,"1,963","95,329",85,164
8,그것: 두 번째 이야기,-1,2019-09-04,"10,170,680",0.7%,"1,213","585,707",96,145
9,극장판 헬로카봇 : 달나라를 구해줘!,-4,2019-09-04,"8,245,900",0.6%,"1,097","559,682",171,198


In [95]:
driver = webdriver.Chrome(path)

driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')
driver.implicitly_wait(4) # 페이지가 로딩될때까지 기다려줍니다

element = driver.find_element_by_id("btn_669")
driver.execute_script("arguments[0].click();", element) # 버튼을 클릭해줄때마다 10개씩 더 나오는 구조
'''
혹시 
WebDriverException: Message: Element is not clickable at point (918, 13). Other element would receive the click: <div class="linkAuchan"></div>
와 같은 에러가 뜰 시에는
driver.find_element_by_id("btn_0").click()  이 코드를 아래의

element = driver.find_element_by_id("btn_572")
driver.execute_script("arguments[0].click();", element)  이 코드로 대체하면 해결할 수 있다.
'''
driver.implicitly_wait(4)

element = driver.find_element_by_id("btn_669")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_669")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)
element = driver.find_element_by_id("btn_669")
driver.execute_script("arguments[0].click();", element)
driver.implicitly_wait(4)

daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'lxml')

daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_669"})

movie_list = daily_boxoffice.find_all("tr")

#간혹 연결을 거부 당하는 경우가 있다
#그럴땐 아래 링크 참조!
#https://www.notion.so/sunnybaik/cc78acee7a0047d68db530eac017f6b9

# 저번시간에 배운 dataframe 을 사용해 보자!
import pandas as pd

daily_movies = {'영화명' : [],
'순위 변동' : [],
'개봉일' : [],
'매출액' : [],
'매출액점유율' : [],
'관객수' : [],
'누적관객수' : [],
'스크린수' : [],
'상영횟수' : []}

for movie in movie_list:
    title_rank = movie.find("td").find_next_sibling("td").text
    daily_movies['영화명'].append(title_rank[:-5].strip())
    daily_movies['순위 변동'].append(title_rank[-4:].strip())
    daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
    daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
 
daily_boxoffice7 = pd.DataFrame(daily_movies) 

daily_boxoffice7

# 순위 변동 칼럼이 계산하기 불편하니 바꿔주자.

def change_rank(rank):
    if rank == "동일":
        rank = 0
    elif rank[-2:] == "하락":
        rank = int(rank[:-2]) * -1
    elif rank[-2:] == "상승":
        rank = int(rank[:-2]) * 1
    return rank

daily_boxoffice7['순위 변동'] = daily_boxoffice7['순위 변동'].map(lambda x: change_rank(x))

        
daily_boxoffice7 # 이렇게 정리한 데이터를 어디다가 저장해야할까 ?

,영화명,순위 변동,개봉일,매출액,매출액점유율,관객수,누적관객수,스크린수,상영횟수
0,나쁜 녀석들: 더 무비,0,2019-09-11,"2,175,129,120",42.5%,"245,297","3,907,202","1,196","5,844"
1,애드 아스트라,0,2019-09-19,"960,976,290",18.8%,"105,570","352,938",835,"3,526"
2,타짜: 원 아이드 잭,0,2019-09-11,"689,953,200",13.5%,"75,948","2,111,784",801,"2,790"
3,예스터데이,0,2019-09-18,"467,709,230",9.1%,"53,538","226,984",647,"2,494"
4,"힘을 내요, 미스터 리",0,2019-09-11,"373,982,300",7.3%,"43,858","1,119,585",597,"1,741"
5,극장판 헬로카봇 : 달나라를 구해줘!,0,2019-09-04,"173,909,330",3.4%,"21,845","558,585",431,644
6,벌새,1,2019-08-29,"32,535,840",0.6%,"3,861","93,366",91,154
7,그것: 두 번째 이야기,-1,2019-09-04,"32,354,470",0.6%,"3,630","584,494",104,161
8,엑시트,0,2019-07-31,"28,623,880",0.6%,"3,421","9,407,848",140,184
9,발명왕 볼트,0,2019-09-19,"25,125,340",0.5%,"3,122","7,386",239,293


In [96]:
import sqlite3

con = sqlite3.connect("dailyboxoffice.db")

con.execute("CREATE TABLE IF NOT EXISTS boxoffice1(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice2(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice3(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice4(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice5(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice6(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
con.execute("CREATE TABLE IF NOT EXISTS boxoffice7(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)")
daily_boxoffice1.to_sql("boxoffice1", con, chunksize=1000, if_exists='replace')
daily_boxoffice2.to_sql("boxoffice2", con, chunksize=1000, if_exists='replace')
daily_boxoffice3.to_sql("boxoffice3", con, chunksize=1000, if_exists='replace')
daily_boxoffice4.to_sql("boxoffice4", con, chunksize=1000, if_exists='replace')
daily_boxoffice5.to_sql("boxoffice5", con, chunksize=1000, if_exists='replace')
daily_boxoffice6.to_sql("boxoffice6", con, chunksize=1000, if_exists='replace')
daily_boxoffice7.to_sql("boxoffice7", con, chunksize=1000, if_exists='replace')

con.commit()

con.close()

In [97]:
import sqlite3
import pandas as pd

con = sqlite3.connect("dailyboxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxoffice1")

data = cursor.fetchall()

In [98]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,0,2019-09-25,"1,730,195,400",33.7%,"201,973","497,845","1,064","4,814"
1,1,양자물리학,1,2019-09-25,"964,521,750",18.8%,"110,099","276,855",772,"2,728"
2,2,나쁜 녀석들: 더 무비,-1,2019-09-11,"957,282,740",18.7%,"109,811","4,353,000","1,000","4,024"
3,3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"467,303,790",9.1%,"52,383","154,415",678,"1,931"
4,4,애드 아스트라,0,2019-09-19,"176,380,740",3.4%,"19,956","470,595",493,919
5,5,퍼펙트맨,1,2019-10-02,"147,407,700",2.9%,"17,495","74,204",481,723
6,6,타짜: 원 아이드 잭,-1,2019-09-11,"107,076,860",2.1%,"11,728","2,202,591",405,907
7,7,더 룸,-1,2019-09-25,"89,743,700",1.7%,"10,108","36,458",163,496
8,8,예스터데이,-1,2019-09-18,"72,460,220",1.4%,"8,237","285,543",317,470
9,9,극장판 헬로카봇 : 달나라를 구해줘!,7,2019-09-04,"51,966,600",1.0%,"6,644","569,399",172,241


In [99]:
cursor.execute("SELECT * FROM boxoffice2")

data = cursor.fetchall()

display(pd.DataFrame(data))

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,0,2019-09-25,"769,141,540",30.3%,"91,325","295,872",979,"4,579"
1,1,나쁜 녀석들: 더 무비,0,2019-09-11,"505,429,190",19.9%,"59,039","4,243,189",921,"3,806"
2,2,양자물리학,0,2019-09-25,"475,273,640",18.7%,"54,820","166,756",658,"2,689"
3,3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"251,645,530",9.9%,"28,391","102,032",668,"1,848"
4,4,애드 아스트라,0,2019-09-19,"107,790,040",4.2%,"12,135","450,639",489,"1,066"
5,5,타짜: 원 아이드 잭,0,2019-09-11,"73,243,300",2.9%,"8,274","2,190,863",429,"1,025"
6,6,더 룸,0,2019-09-25,"56,364,900",2.2%,"6,356","26,350",157,562
7,7,예스터데이,1,2019-09-18,"47,211,980",1.9%,"5,605","277,306",337,525
8,8,레플리카,-1,2019-09-25,"43,191,260",1.7%,"5,060","18,625",339,774
9,9,"힘을 내요, 미스터 리",0,2019-09-11,"23,519,940",0.9%,"3,390","1,158,974",222,319


In [100]:
cursor.execute("SELECT * FROM boxoffice3")

data = cursor.fetchall()
display(pd.DataFrame(data))

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,0,2019-09-25,"521,422,010",27.4%,"66,273","204,547",977,"4,390"
1,1,나쁜 녀석들: 더 무비,0,2019-09-11,"366,659,350",19.3%,"46,264","4,184,150",891,"3,488"
2,2,양자물리학,0,2019-09-25,"334,299,980",17.6%,"40,798","111,936",669,"2,665"
3,3,원스 어폰 어 타임... 인 할리우드,0,2019-09-25,"181,361,350",9.5%,"22,376","73,641",640,"1,716"
4,4,애드 아스트라,0,2019-09-19,"82,494,370",4.3%,"9,950","438,504",484,"1,103"
5,5,타짜: 원 아이드 잭,0,2019-09-11,"62,133,940",3.3%,"7,722","2,182,589",424,"1,018"
6,6,더 룸,0,2019-09-25,"41,016,200",2.2%,"4,938","19,994",144,572
7,7,레플리카,1,2019-09-25,"38,023,670",2.0%,"4,831","13,565",344,838
8,8,예스터데이,-1,2019-09-18,"33,855,500",1.8%,"4,327","271,701",338,541
9,9,"힘을 내요, 미스터 리",0,2019-09-11,"24,129,260",1.3%,"3,710","1,155,584",215,305


In [101]:
cursor.execute("SELECT * FROM boxoffice4")

data = cursor.fetchall()
display(pd.DataFrame(data))

,0,1,2,3,4,5,6,7,8,9
0,0,장사리 : 잊혀진 영웅들,7,2019-09-25,"770,367,470",31.0%,"129,938","138,274","1,065","4,381"
1,1,나쁜 녀석들: 더 무비,-1,2019-09-11,"531,802,230",21.4%,"89,092","4,137,886",881,"3,229"
2,2,양자물리학,4,2019-09-25,"323,242,370",13.0%,"53,451","71,138",731,"2,675"
3,3,원스 어폰 어 타임... 인 할리우드,7,2019-09-25,"289,132,600",11.6%,"47,987","51,265",670,"1,648"
4,4,애드 아스트라,-3,2019-09-19,"118,937,110",4.8%,"19,165","428,554",520,"1,159"
5,5,타짜: 원 아이드 잭,-3,2019-09-11,"88,376,020",3.6%,"14,507","2,174,867",516,"1,083"
6,6,더 룸,7,2019-09-25,"60,565,900",2.4%,"9,912","15,056",159,587
7,7,예스터데이,-3,2019-09-18,"55,069,000",2.2%,"8,953","267,374",374,607
8,8,레플리카,New,2019-09-25,"52,701,990",2.1%,"8,544","8,734",360,871
9,9,"힘을 내요, 미스터 리",-4,2019-09-11,"34,873,770",1.4%,"5,965","1,151,874",256,359


In [102]:
cursor.execute("SELECT * FROM boxoffice5")

data = cursor.fetchall()
display(pd.DataFrame(data))


,0,1,2,3,4,5,6,7,8,9
0,0,나쁜 녀석들: 더 무비,0,2019-09-11,"526,512,270",32.3%,"65,944","4,048,794","1,114","5,235"
1,1,애드 아스트라,0,2019-09-19,"217,903,930",13.4%,"26,323","409,389",787,"3,179"
2,2,타짜: 원 아이드 잭,0,2019-09-11,"181,626,290",11.1%,"22,523","2,160,360",777,"2,661"
3,3,퍼펙트맨,2,2019-10-02,"159,523,000",9.8%,"21,932","55,272",147,164
4,4,예스터데이,-1,2019-09-18,"117,676,790",7.2%,"14,971","258,421",630,"2,161"
5,5,"힘을 내요, 미스터 리",-1,2019-09-11,"100,037,300",6.1%,"13,612","1,145,909",584,"1,642"
6,6,양자물리학,2,2019-09-25,"49,990,000",3.1%,"8,570","17,687",64,65
7,7,장사리 : 잊혀진 영웅들,5,2019-09-25,"33,219,000",2.0%,"3,644","8,336",17,17
8,8,가장 보통의 연애,New,2019-10-02,"30,123,000",1.8%,"3,347","3,748",15,18
9,9,그것: 두 번째 이야기,-1,2019-09-04,"28,255,000",1.7%,"3,278","588,985",109,160


In [103]:
cursor.execute("SELECT * FROM boxoffice6")

data = cursor.fetchall()
display(pd.DataFrame(data))

,0,1,2,3,4,5,6,7,8,9
0,0,나쁜 녀석들: 더 무비,0,2019-09-11,"609,349,940",41.4%,"75,648","3,982,850","1,084","5,373"
1,1,애드 아스트라,0,2019-09-19,"250,433,040",17.0%,"30,128","383,066",799,"3,291"
2,2,타짜: 원 아이드 잭,0,2019-09-11,"210,394,610",14.3%,"26,053","2,137,837",748,"2,713"
3,3,예스터데이,0,2019-09-18,"131,507,040",8.9%,"16,466","243,450",614,"2,277"
4,4,"힘을 내요, 미스터 리",0,2019-09-11,"95,046,390",6.5%,"12,712","1,132,297",558,"1,668"
5,5,퍼펙트맨,2,2019-10-02,"23,418,000",1.6%,"2,640","33,340",13,13
6,6,더 룸,7,2019-09-25,"18,560,000",1.3%,"2,320","4,848",11,11
7,7,벌새,-1,2019-08-29,"15,188,600",1.0%,"1,963","95,329",85,164
8,8,그것: 두 번째 이야기,-1,2019-09-04,"10,170,680",0.7%,"1,213","585,707",96,145
9,9,극장판 헬로카봇 : 달나라를 구해줘!,-4,2019-09-04,"8,245,900",0.6%,"1,097","559,682",171,198


In [104]:
cursor.execute("SELECT * FROM boxoffice7")

data = cursor.fetchall()
display(pd.DataFrame(data))

,0,1,2,3,4,5,6,7,8,9
0,0,나쁜 녀석들: 더 무비,0,2019-09-11,"2,175,129,120",42.5%,"245,297","3,907,202","1,196","5,844"
1,1,애드 아스트라,0,2019-09-19,"960,976,290",18.8%,"105,570","352,938",835,"3,526"
2,2,타짜: 원 아이드 잭,0,2019-09-11,"689,953,200",13.5%,"75,948","2,111,784",801,"2,790"
3,3,예스터데이,0,2019-09-18,"467,709,230",9.1%,"53,538","226,984",647,"2,494"
4,4,"힘을 내요, 미스터 리",0,2019-09-11,"373,982,300",7.3%,"43,858","1,119,585",597,"1,741"
5,5,극장판 헬로카봇 : 달나라를 구해줘!,0,2019-09-04,"173,909,330",3.4%,"21,845","558,585",431,644
6,6,벌새,1,2019-08-29,"32,535,840",0.6%,"3,861","93,366",91,154
7,7,그것: 두 번째 이야기,-1,2019-09-04,"32,354,470",0.6%,"3,630","584,494",104,161
8,8,엑시트,0,2019-07-31,"28,623,880",0.6%,"3,421","9,407,848",140,184
9,9,발명왕 볼트,0,2019-09-19,"25,125,340",0.5%,"3,122","7,386",239,293
